In [11]:
import os
import glob
import json, fire, re
from pathlib import Path
import io

def is_export(cell):
    if cell['cell_type'] != 'code': 
        return False
    src = cell['source']
    if len(src) == 0 or len(src[0]) < 7: 
        return False
    return re.match(r'^\s*#\s*export\s*$', src[0], re.IGNORECASE) is not None
        
def notebook2script(f_name):
    "Finds cells starting with `#export` and puts them into a new module"
    f_name = Path(f_name)
    main_dic = json.load(open(f_name,'r',encoding="utf-8"))
    code_cells = [c for c in main_dic['cells'] if is_export(c)]
    module = '# ---------------------------------------------\n' +\
             '# | THIS FILE WAS AUTOGENERATED! DO NOT EDIT! |\n' +\
             '# ---------------------------------------------\n' +\
            f'# edit notebooks/{f_name.name} and run generate_all.py\n\n'
                
    if not (Path('scripts')).exists(): 
        ('scripts').mkdir()
    output_path = 'scripts/' + f'{f_name.stem[3:]}.py'
    
    for cell in code_cells: 
        module += ''.join(cell['source'][1:]) + '\n\n'
    module = re.sub(r' +$', '', module, flags=re.MULTILINE)
    
    with io.open(output_path, "w", encoding="utf-8") as f:
        f.write(module[:-2])
        
    print(f"Converted {f_name} to {output_path}")

    
files = glob.glob('scripts/*')
for f_name in files:
    if f_name.endswith('py'):
        os.remove(f_name)

for f_name in glob.glob('notebooks/*'):
    if f_name.endswith('ipynb'):
        notebook2script(f_name)

Converted notebooks/03_callback.ipynb to scripts/callback.py
Converted notebooks/07_learning_rate_search.ipynb to scripts/learning_rate_search.py
Converted notebooks/02_training.ipynb to scripts/training.py
Converted notebooks/05_hyper_param_scheduling.ipynb to scripts/hyper_param_scheduling.py
Converted notebooks/01_sequential_model.ipynb to scripts/sequential_model.py
Converted notebooks/04_stats_logging.ipynb to scripts/stats_logging.py
Converted notebooks/00_operations.ipynb to scripts/operations.py
Converted notebooks/06_early_stopping.ipynb to scripts/early_stopping.py
